# GNN stream adj,fea and weight matrix multiplication with AXI master interfaces on PYNQ

## Import and download the HLS Overlay

In [133]:
#import os
#base_dir=os.getcwd()
#filename="/home/xilinx/pynq/overlays/gnn/gnn.bit";
#bitname=os.path.join(base_dir, filename)
#print(bitname)
#import torch

In [134]:
from pynq import Overlay
ol = Overlay("gnn_all.bit")

## DATA SET SELECTION. Execute only cell for the data you want to test . e.g. CITESEER


In [6]:
adj_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/mol_adj.txt'
fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/mol_feat.txt'
#fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/mol_feat_dense.txt'
w_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/mol_weights.txt'
N_adj = 2273  # number of nodes
M_fea = 7  # number of input features
P_w = 32  # number of features in the hidden layer
NNZ_adj = 5028  # number of non-zero values of adjacency
NNZ_fea = 2273  # number of non-zero values of feature
ADJ_THREADS = 1

In [110]:
adj_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/cora_adj.txt'
fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/cora_feat.txt'
w_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/cora_weights.txt'
N_adj = 2708  # number of nodes
M_fea = 1433  # number of input features
P_w = 16  # number of features in the hidden layer
NNZ_adj = 13264  # number of non-zero values of adjacency
NNZ_fea = 49216  # number of non-zero values of feature
ADJ_THREADS = 1

In [135]:
adj_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/citeseer_adj.txt'
fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/citeseer_feat.txt'
w_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/citeseer_weights.txt'
N_adj = 3327  # number of nodes
M_fea = 3703  # number of input features
P_w = 16  # number of features in the hidden layer
NNZ_adj = 12431  # number of non-zero values of adjacency
NNZ_fea = 105165  # number of non-zero values of feature
ADJ_THREADS = 1

In [136]:
#adj_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/pubmed_adj.txt'
#fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/pubmed_feat.txt'
#w_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/pubmed_weights.txt'
#N_adj = 19717  # number of nodes
#M_fea = 500  # number of input features
#P_w = 16  # number of features in the hidden layer
#NNZ_adj = 108365  # number of non-zero values of adjacency
#NNZ_fea = 988031  # number of non-zero values of feature
#ADJ_THREADS = 1

## Examine the design 

In [137]:
ol.ip_dict

{'axi_intc_0': {'fullpath': 'axi_intc_0',
  'type': 'xilinx.com:ip:axi_intc:4.1',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 2684354560,
  'mem_id': 's_axi',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {'intr': {'controller': 'axi_intc_0',
    'index': 0,
    'fullpath': 'axi_intc_0/intr'}},
  'parameters': {'C_FAMILY': 'zynquplus',
   'C_INSTANCE': 'design_1_axi_intc_0_0',
   'C_S_AXI_ADDR_WIDTH': '9',
   'C_S_AXI_DATA_WIDTH': '32',
   'C_NUM_INTR_INPUTS': '1',
   'C_NUM_SW_INTR': '0',
   'C_KIND_OF_INTR': '0xfffffffe',
   'C_KIND_OF_EDGE': '0xFFFFFFFF',
   'C_KIND_OF_LVL': '0xFFFFFFFF',
   'C_ASYNC_INTR': '0xFFFFFFFE',
   'C_NUM_SYNC_FF': '2',
   'C_ADDR_WIDTH': '32',
   'C_IVAR_RESET_VALUE': '0x0000000000000010',
   'C_ENABLE_ASYNC': '0',
   'C_HAS_IPR': '1',
   'C_HAS_SIE': '1',
   'C_HAS_CIE': '1',
   'C_HAS_IVR': '1',
   'C_HAS_ILR': '0',
   'C_IRQ_IS_LEVEL': '1',
   'C_IRQ_ACTIVE': '0x1',
   'C_DISABLE_SYNCHRONIZERS': '0',
   'C_MB_CLK_NO

## Create an alias for the IP

In [138]:
my_ip = ol.mmult_top_0
#my_monitor = ol.my_monitor_0

## Check the register map for the HLS IP

You should see the registers that we need to write values to:

* CTRL - the control register
* A_offset_1 - the lower 32-bits of the start address for the A matrix
* B_offset_1 - the lower 32-bits of the start address for the B matrix
* C_offset_1 - the lower 32-bits of the start address for the C matrix

In [139]:
my_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  gemm_mode = Register(gemm_mode=write-only, RESERVED=write-only),
  relu = Register(relu=write-only, RESERVED=write-only),
  quantized_multiplier_offset_1 = Register(quantized_multiplier_offset=write-only),
  quantized_multiplier_offset_2 = Register(quantized_multiplier_offset=write-only),
  shift_offset_1 = Register(shift_offset=write-only),
  shift_offset_2 = Register(shift_offset=write-only),
  bias_offset_1 = Register(bias_offset=write-only),
  bias_offset_2 = Register(bias_offset=write-only),
  bias_count = Register(bias_count=write-only),
  profiling_offset_1 = Register(profiling_offset=write-only),
  profiling_offset_2 = Register(profiling_offset=write-only),

In [140]:
my_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  gemm_mode = Register(gemm_mode=write-only, RESERVED=write-only),
  relu = Register(relu=write-only, RESERVED=write-only),
  quantized_multiplier_offset_1 = Register(quantized_multiplier_offset=write-only),
  quantized_multiplier_offset_2 = Register(quantized_multiplier_offset=write-only),
  shift_offset_1 = Register(shift_offset=write-only),
  shift_offset_2 = Register(shift_offset=write-only),
  bias_offset_1 = Register(bias_offset=write-only),
  bias_offset_2 = Register(bias_offset=write-only),
  bias_count = Register(bias_count=write-only),
  profiling_offset_1 = Register(profiling_offset=write-only),
  profiling_offset_2 = Register(profiling_offset=write-only),

In [141]:
print("Allocate some memory")

##The start address for this memory will be passed to the HLS IP. The HLS IP will read data from this memory and write results back to the same locations. 

##We can initialise the memory 

Allocate some memory


In [142]:
from pynq import allocate
import numpy as np 
quantized_multiplier_buffer  = allocate(1024, dtype=np.int32)
bias_buffer  = allocate(1024, dtype=np.int32)
shift_buffer  = allocate(1024, dtype=np.int32)
profiling_buffer  = allocate(16, dtype=np.int64)
rowPtr_fea_buffer = allocate(N_adj+1, dtype=np.int32)
columnIndex_fea_buffer = allocate(NNZ_fea, dtype=np.int32)
#values_fea_buffer = allocate(NNZ_fea, dtype=np.float16)
values_fea_buffer = allocate(200000, dtype=np.float16)
rowPtr_adj_buffer = allocate(N_adj+1, dtype=np.int32)
columnIndex_adj_buffer = allocate(NNZ_adj, dtype=np.int32)
values_adj_buffer = allocate(NNZ_adj, dtype=np.float16)
B_buffer = allocate(shape=(P_w,M_fea), dtype=np.float16)
#B_buffer = allocate((M_fea*P_w), dtype=np.float32)
#D_buffer = allocate((N_adj*P_w), dtype=np.float32)
D_buffer = allocate(shape=(N_adj,P_w), dtype=np.float16)

In [143]:
## Initialize the D and B buffers

print("Load weight data")

Load weight data


In [144]:
fin = open(w_name, 'rb')
#w_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.float32)
w = np.genfromtxt(fin, delimiter=',', dtype=np.float16)
for i in range(M_fea): 
    for j in range(P_w):
            B_buffer[j][i] = w[i][j]
            #B_buffer[i*P_w+j] = w[i][j] * (1 << FRAC_BITS_IN)
            #B_buffer[i*P_w+j] = 255

In [145]:
#B_buffer=B_buffer.transpose() #we transpose weights so hardware can read column as contiguos data

In [146]:
B_buffer

PynqBuffer([[ 0.01738  ,  0.03238  , -0.000407 , ..., -0.001362 ,
             -0.       , -0.000456 ],
            [-0.01135  , -0.0333   , -0.000504 , ..., -0.001169 ,
             -0.       , -0.006878 ],
            [-0.02078  ,  0.0268   ,  0.000892 , ..., -0.003405 ,
              0.       , -0.0085   ],
            ...,
            [-0.018    ,  0.01904  ,  0.001408 , ...,  0.001011 ,
              0.       ,  0.007812 ],
            [ 0.01994  ,  0.05753  ,  0.0002897, ..., -0.00202  ,
             -0.       , -0.002542 ],
            [-0.001315 , -0.02975  , -0.0001557, ...,  0.003527 ,
              0.       ,  0.010315 ]], dtype=float16)

In [147]:
for i in range(N_adj):
    for j in range(P_w):
        D_buffer[i][j] = 1

In [148]:
np.amin(B_buffer)

PynqBuffer(-0.1556, dtype=float16)

In [149]:
np.amax(B_buffer)

PynqBuffer(0.265, dtype=float16)

## Read Sparse data



In [150]:
with open(adj_name, 'rb') as fin:
    rowPtr_adj_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #print(rowPtr_adj_row[3327])
    #for i in range(N_adj+1):
        #print(rowPtr_adj_row[i])
        #rowPtr_adj_buffer[i] = rowPtr_adj_row[i]
    rowPtr_adj_buffer[:] = rowPtr_adj_row[:]
    columnIndex_adj_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #for i in range(NNZ_adj):
        #columnIndex_adj_buffer[i] = columnIndex_adj_row[i]
    columnIndex_adj_buffer[:] = columnIndex_adj_row[:]
    values_adj_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.float16)
    #for i in range(NNZ_adj):
    values_adj_buffer[:] = values_adj_row[:]
    #values_adj_buffer[:] = values_adj_row[:]
        #values_adj_buffer[i] = 255

In [151]:
with open(fea_name, 'rb') as fin:
    rowPtr_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #for i in range(N_adj+1):
        #rowPtr_fea_buffer[i] = rowPtr_fea_row[i]
        #print(rowPtr_fea_buffer[i])
    rowPtr_fea_buffer[:] = rowPtr_fea_row[:]
    columnIndex_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #for i in range(NNZ_fea):
    #    columnIndex_fea_buffer[i] = columnIndex_fea_row[i]
    columnIndex_fea_buffer[:] = columnIndex_fea_row[:]
    #columnIndex_fea_buffer = columnIndex_fea_row
    values_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.float16)
    #for i in range(NNZ_fea):
        #values_fea_buffer[i] = 0
    values_fea_buffer[0:NNZ_fea] = values_fea_row[:]
    #values_fea_buffer[0:2273] = values_fea_row[:]

In [152]:
#get dense matrix if you are testing gemm mode (dense) with my_ip.register_map.gemm_mode=1 (see code below)
#otherwise do not execute this cell to test normal sparse mode
#from scipy.sparse import csr_matrix
#import pandas as pd
#values_fea_row = values_fea_row.astype(np.float32)
#csrMatrix_fea = csr_matrix((values_fea_row , columnIndex_fea_row, rowPtr_fea_row), shape=(N_adj, M_fea))
#dense_fea = csrMatrix_fea.todense()
#df = pd.DataFrame(dense_fea) #convert to a dataframe
#df.to_csv("./fea_values.txt",index=False,header=False) #save to file
#dense_fea = dense_fea.reshape(1,N_adj*M_fea)
#data = dense_fea.astype(np.float16)
#values_fea_buffer[0:N_adj*M_fea] = data[:]

In [153]:
print("rowPtr_fea_buffer physical address {}".format(hex(rowPtr_fea_buffer.physical_address)))
print("columnIndex_fea_buffer physical address {}".format(hex(columnIndex_fea_buffer.physical_address)))
print("values_fea_buffer physical address {}".format(hex(values_fea_buffer.physical_address)))
print("rowPtr_adj_buffer physical address {}".format(hex(rowPtr_adj_buffer.physical_address)))
print("columnIndex_adj_buffer physical address {}".format(hex(columnIndex_adj_buffer.physical_address)))
print("quantized_multiplier physical address {}".format(hex(quantized_multiplier_buffer.physical_address)))
print("bias physical address {}".format(hex(bias_buffer.physical_address)))
print("shift physical address {}".format(hex(shift_buffer.physical_address)))
print("B_buffer physical address {}".format(hex(B_buffer.physical_address)))
print("D_buffer physical address {}".format(hex(D_buffer.physical_address)))
print("profiling physical address {}".format(hex(profiling_buffer.physical_address)))

rowPtr_fea_buffer physical address 0x788e4000
columnIndex_fea_buffer physical address 0x79100000
values_fea_buffer physical address 0x79180000
rowPtr_adj_buffer physical address 0x788e8000
columnIndex_adj_buffer physical address 0x789d0000
quantized_multiplier physical address 0x73961000
bias physical address 0x73962000
shift physical address 0x73963000
B_buffer physical address 0x79060000
D_buffer physical address 0x79200000
profiling physical address 0x73964000


## Configure the IP

Write the values of the memory address for all the buffers.
The run kernel starts the IP block and checks the done bit to see when it goes high. 

In [154]:
my_ip.register_map.B_offset_1 = B_buffer.physical_address
my_ip.register_map.D1_offset_1 = D_buffer.physical_address
my_ip.register_map.D2_offset_1 = D_buffer.physical_address+P_w*N_adj/ADJ_THREADS
my_ip.register_map.D3_offset_1 = D_buffer.physical_address+P_w*2*N_adj/ADJ_THREADS
my_ip.register_map.D4_offset_1 = D_buffer.physical_address+P_w*3*N_adj/ADJ_THREADS
my_ip.register_map.rowPtr_fea1_offset_1 = rowPtr_fea_buffer.physical_address
my_ip.register_map.rowPtr_fea2_offset_1 = rowPtr_fea_buffer.physical_address
my_ip.register_map.rowPtr_fea3_offset_1 = rowPtr_fea_buffer.physical_address
my_ip.register_map.rowPtr_fea4_offset_1 = rowPtr_fea_buffer.physical_address
#my_ip.register_map.rowPtr_fea_offset_1 = 0
my_ip.register_map.columnIndex_fea1_offset_1 =columnIndex_fea_buffer.physical_address
my_ip.register_map.columnIndex_fea2_offset_1 =columnIndex_fea_buffer.physical_address 
my_ip.register_map.columnIndex_fea3_offset_1 =columnIndex_fea_buffer.physical_address 
my_ip.register_map.columnIndex_fea4_offset_1 =columnIndex_fea_buffer.physical_address 
#my_ip.register_map.columnIndex_fea_offset_1 =0
my_ip.register_map.values_fea1_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.values_fea2_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.values_fea3_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.values_fea4_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.rowPtr_adj1_offset_1 = rowPtr_adj_buffer.physical_address 
my_ip.register_map.rowPtr_adj2_offset_1 = rowPtr_adj_buffer.physical_address 
my_ip.register_map.rowPtr_adj3_offset_1 = rowPtr_adj_buffer.physical_address
my_ip.register_map.rowPtr_adj4_offset_1 = rowPtr_adj_buffer.physical_address
#my_ip.register_map.rowPtr_adj_offset_1 = 0
my_ip.register_map.columnIndex_adj1_offset_1 = columnIndex_adj_buffer.physical_address 
my_ip.register_map.columnIndex_adj2_offset_1 = columnIndex_adj_buffer.physical_address
my_ip.register_map.columnIndex_adj3_offset_1 = columnIndex_adj_buffer.physical_address 
my_ip.register_map.columnIndex_adj4_offset_1 = columnIndex_adj_buffer.physical_address 
#my_ip.register_map.columnIndex_adj_offset_1 = 0
my_ip.register_map.values_adj1_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.values_adj2_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.values_adj3_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.values_adj4_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.quantized_multiplier_offset_1 = quantized_multiplier_buffer.physical_address
my_ip.register_map.bias_offset_1  = bias_buffer.physical_address
my_ip.register_map.shift_offset_1  = shift_buffer.physical_address
my_ip.register_map.profiling_offset_1  = profiling_buffer.physical_address
my_ip.register_map.M_fea=M_fea
my_ip.register_map.N_adj=N_adj
my_ip.register_map.M_adj=N_adj
my_ip.register_map.P_w=P_w
my_ip.register_map.relu=0
my_ip.register_map.gemm_mode=0
my_ip.register_map.bias_count=0
my_ip.register_map.nnz_fea=NNZ_fea
my_ip.register_map.array_c_adjust=N_adj

In [155]:
def run_kernel():
    my_ip.register_map.CTRL.AP_START=1
    kernel_done = my_ip.register_map.CTRL.AP_DONE
    while kernel_done == 0:
            kernel_done = my_ip.register_map.CTRL.AP_DONE

In [156]:
## Start the IP and time its execution time
print("Running..")

Running..


In [158]:
%timeit run_kernel() #citeseer 1t1t2c takes 4.65 ms
#run_kernel()

4.65 ms ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Check the contents of the memory to see if they look OK

In [159]:
result=np.zeros(shape=(N_adj,P_w), dtype=np.float16)
result[:] = D_buffer[:]
result[:] = result[:]

In [160]:
with np.printoptions(threshold=np.inf,suppress=True):
    print(result[0])

[0.001968 0.4778   0.5967   0.03113  0.002258 0.5645   0.5312   0.01105
 0.1146   0.716    0.0995   0.1425   0.653    0.1676   0.001312 0.07336 ]


In [161]:
result

array([[ 1.968e-03,  4.778e-01,  5.967e-01, ...,  1.676e-01,  1.312e-03,
         7.336e-02],
       [ 6.787e-01, -5.923e-02,  7.480e-01, ...,  8.472e-01,  6.948e-01,
         9.198e-02],
       [-9.766e-04,  6.626e-01,  6.104e-05, ..., -2.441e-04,  9.155e-05,
         7.568e-01],
       ...,
       [ 1.338e-01,  3.181e-01,  4.409e-01, ...,  4.189e-01,  9.131e-02,
         3.503e-01],
       [ 3.132e-01,  5.188e-03,  5.020e-01, ...,  6.592e-01,  4.028e-01,
         2.485e-01],
       [ 1.831e-04,  4.583e-01,  4.126e-02, ...,  3.662e-04,  7.983e-02,
         4.976e-01]], dtype=float16)

In [162]:
print("fifo_full_0 (frontend process fast)=",profiling_buffer[0])
print("fifo_empty_0 (frontend process slow)=",profiling_buffer[3])
print("fifo_read_0 (data read process count)=",profiling_buffer[6])
print("fifo_write_0 (data write process count)=",profiling_buffer[9])
print("fifo_cycle_0 (data activation process count)=",profiling_buffer[12])

fifo_full_0 (frontend process fast)= 0
fifo_empty_0 (frontend process slow)= 0
fifo_read_0 (data read process count)= 0
fifo_write_0 (data write process count)= 0
fifo_cycle_0 (data activation process count)= 0


In [163]:
print("fifo_full_2 (backend process slow)=",profiling_buffer[2])
print("fifo_empty_2 (backend process fast)=",profiling_buffer[5])
print("fifo_read_2 (data read process count)=",profiling_buffer[8])
print("fifo_write_2 (data write process count)=",profiling_buffer[11])
print("fifo_cycle_2 (data activation process count)=",profiling_buffer[14])

fifo_full_2 (backend process slow)= 0
fifo_empty_2 (backend process fast)= 0
fifo_read_2 (data read process count)= 0
fifo_write_2 (data write process count)= 0
fifo_cycle_2 (data activation process count)= 0


In [164]:
#my_monitor.register_map


In [165]:
#my_ip.register_map.CTRL
#fea_vfifo_full=int(my_monitor.register_map.reg1)
#fea_vfifo_empty=int(my_monitor.register_map.reg2)
#adj_vfifo_full=int(my_monitor.register_map.reg3)
#adj_vfifo_empty=int(my_monitor.register_map.reg4)
#active=int(my_monitor.register_map.reg0)

In [166]:
#print("vfifo fea empty rate ",fea_vfifo_empty/active)

In [167]:
#print("vfifo fea full rate ",fea_vfifo_full/active)

In [168]:
#print("vfifo adj empty rate ",adj_vfifo_empty/active)

In [169]:
#print("vfifo adj full rate ",adj_vfifo_full/active)

In [170]:
#print("Execution time in ms",1000*active/250000000)

In [171]:
#my_ip.register_map.CTRL.AUTO_RESTART=1

# Replicate in software to compare performance

In [172]:
quantized_multiplier_buffer.freebuffer()
bias_buffer.freebuffer()
shift_buffer.freebuffer()
profiling_buffer.freebuffer()
rowPtr_fea_buffer.freebuffer()
columnIndex_fea_buffer.freebuffer()
values_fea_buffer.freebuffer()
rowPtr_adj_buffer.freebuffer()
columnIndex_adj_buffer.freebuffer()
values_adj_buffer.freebuffer()
B_buffer.freebuffer()
D_buffer.freebuffer()

In [173]:
from scipy.sparse import csr_matrix
csrMatrix_adj = csr_matrix((values_adj_row , columnIndex_adj_row, rowPtr_adj_row), shape=(N_adj, N_adj))
csrMatrix_fea = csr_matrix((values_fea_row , columnIndex_fea_row, rowPtr_fea_row), shape=(N_adj, M_fea))

In [174]:
w.shape

(3703, 21)

In [175]:
#%timeit csrMatrix_adjfeaw = csrMatrix_adj@(csrMatrix_fea@w)
csrMatrix_adjfeaw = csrMatrix_adj@(csrMatrix_fea@w)

In [176]:
csrMatrix_adjfeaw.shape

(3327, 21)

In [177]:
with np.printoptions(threshold=np.inf,suppress=True):
    print(csrMatrix_adjfeaw[0])

[0.0019877  0.47759533 0.5971749  0.03115976 0.00219059 0.5643878
 0.530963   0.01102665 0.11454511 0.7159475  0.09950522 0.14244223
 0.6527748  0.16745949 0.00133669 0.073318   0.12169862 0.15521336
 0.60524035 0.4921999  0.4949317 ]


In [95]:
#np.allclose(result, csrMatrix_adjfeaw,atol=0.1)

In [96]:
#support = torch.matmul(csrMatrix_fea,w)
#output = torch.spmm(csrMatrix_adj, support)
#support = torch.matmul(csrMatrix_fea,w)